### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?



In [17]:
# Here is a code for gradient descent without line search
import numpy as np
import math as m
import torch as t
from torch.autograd import Variable
from line_search_bt import *
T=20

a_water=np.array([8.07131,1730.63,233.426])
a_dioxane=np.array([7.43155,1554.679,240.337])

x1=np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
x2=1-x1

p=np.array([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])

def p_specific(a,T):
   return 10**(a[0]-((a[1])/(T+a[2])))

pwater=((p_specific(a_water,T)))
pdioxane=((p_specific(a_dioxane,T)))

x = Variable(t.tensor([1.0, 1.0]), requires_grad=True)

# Fix the step size
a = 0.001

# Start gradient descent
for i in range(100):  # TODO: change the termination criterion
    for i in range(0,len(x1)):
        
        loss = (((x1[i]*pwater*t.exp(x[0]*((x[1]*x2[i])/(x[0]*x1[i]+x[1]*x2[i]))**2)) + (x2[i]*pdioxane*t.exp( x[1]*((x[0]*x1[i])/(x[0]*x1[i]+x[1]*x2[i]))**2))) - p[i])**2
    
        loss.backward()
    x.grad.numpy()
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        
        x -= a * x.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.grad.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())

for i in range(0,len(p)):
   pcheck = ((x1[i]*pwater*m.exp(x[0]*((x[1]*x2[i])/(x[0]*x1[i]+x[1]*x2[i]))**2)) + (x2[i]*pdioxane*m.exp( x[1]*((x[0]*x1[i])/(x[0]*x1[i]+x[1]*x2[i]))**2)))
   print("p=",p[i], "&", "p model=",pcheck, "Error" , p[i]-pcheck)

[1.9584177 1.6891866]
0.00071549066
p= 28.1 & p model= 28.824099527405245 Error -0.7240995274052437
p= 34.4 & p model= 34.64430983581709 Error -0.24430983581709143
p= 36.7 & p model= 36.452964624211475 Error 0.24703537578852774
p= 36.9 & p model= 36.86731249815574 Error 0.03268750184425784
p= 36.8 & p model= 36.87400708532342 Error -0.07400708532342293
p= 36.7 & p model= 36.74983136261855 Error -0.049831362618547814
p= 36.5 & p model= 36.390446913894365 Error 0.10955308610563463
p= 35.4 & p model= 35.38482080704233 Error 0.015179192957667453
p= 32.9 & p model= 32.94778400610805 Error -0.04778400610805278
p= 27.7 & p model= 27.730002077430925 Error -0.03000207743092531
p= 17.5 & p model= 17.47325208459706 Error 0.026747915402939526


### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).



In [1]:
import numpy as np 
import sklearn.gaussian_process as gp
def func(x):
    return (4-2.1*x[0]**+(x[0]**4/3))*x[0]**2+x[0]*x[1]+(-4+4*x[1]**2)*x[1]**2
niters=100
x=[0,0]

def bayesian_optimization(n_iters,func(x), xp, yp):

      kernel = gp.kernels.Matern()
  model = gp.GaussianProcessRegressor(kernel=kernel,alpha=1e-4,n_restarts_optimizer=10,normalize_y=True)
                               
  for i in range(n_iters):
    # Update our belief of the loss function
    model.fit(xp, yp)

    # sample_next_hyperparameter is a method that computes the arg
    # max of the acquisition function
    next_sample = sample_next_hyperparameter(model, yp)

    # Evaluate the loss for the new hyperparameters
    next_loss = sample_loss(next_sample)

    # Update xp and yp


SyntaxError: invalid syntax (Temp/ipykernel_28488/3315217098.py, line 5)